# Python y Snowflake

1. Crea conexion a Snowflake
2. Consulta una tabla en Snowflake
3. Convierte el resultado en un DataFrame

In [1]:
import snowflake.connector
import pandas as pd

Cargar variables de entorno

In [2]:
def load_credentials():

    from dotenv import load_dotenv
    from os import environ

    def load_secrets():
        credentials = dict()
        credentials["user"] = environ.get("snowflake_user")
        credentials["password"] = environ.get("snowflake_password")
        credentials["role"] = environ.get("snowflake_role")
        credentials["account"] = environ.get("snowflake_account")
        credentials["warehouse"] = environ.get("snowflake_wh")
        for k,v in credentials.items():
            if v==None:
                e = f"value for [{k}] is None."
                raise Exception(e)
        return credentials

    try:
        credentials = load_secrets()
        print("Variables cargadadas desde el entorno local.")
    except:
        load_dotenv()
        credentials = load_secrets()
        print("Variables cargadadas desde [.env]")
    
    credentials.update({"host": f'{credentials["account"]}.us-east-1.snowflakecomputing.com'})
    
    return credentials

In [3]:
credentials = load_credentials()
credentials.keys()

Variables cargadadas desde [.env]


dict_keys(['user', 'password', 'role', 'account', 'warehouse', 'host'])

1. Crear conexion

In [4]:
conn = snowflake.connector.connect(**credentials)
conn

2. Realizar Querys

In [5]:
database='SNOWFLAKE'
schema='ref'
tb = 'divipola'
sql = f"select * from {database}.{schema}.{tb};"
print(f"Consultar tabla [{tb}] de snowflake.")

Consultar tabla [divipola] de snowflake.


3. Retorna el resultado de un query en un Df.

In [6]:
def run_sql_query(sql:str, conn):
    cur = conn.cursor()
    cur.execute(sql)
    df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
    print("df.shape = ", df.shape)
    return df

In [7]:
%%time
df = run_sql_query(sql=sql, conn=conn)
df.head()

df.shape =  (1122, 4)
CPU times: total: 78.1 ms
Wall time: 423 ms


,DEPARTAMENTO_COD,DEPARTAMENTO,DIVIPOLA,MUNICIPIO
0,11,"Bogotá, D.C.",11001,"Bogotá, D.C."
1,15,Boyacá,15226,Cuítiva
2,15,Boyacá,15518,Pajarito
3,25,Cundinamarca,25394,La Palma
4,47,Magdalena,47605,Remolino
